# Preprocessing the Dataset

- This notebook assumes that you already have a dataset generated in BOP Format in /dataset directory.
- Once the preprocessing is completed you can use the custom dataloader in the /lib directory for training etc
- Here we generate Keypoints, Corners and Pointcloud Features

In [1]:
import os
import open3d as o3d
import pickle as pkl
import json
import copy
import torch
import numpy as np

root_path = "/media/kulunu/Elements SE/Datasets/Nema17_reducer_dataset"
data_path = root_path + "/stage_1"

seg_format = '.npy'
img_id_leading_zeros = 0
dpt_format = ".png"
rgb_format = ".png"

# data were rendered like this
# python3 render_icp_data.py -m Nema17 sun_gear housing carrier cover -d Nema17_reducer -s 1
# so object ids :
# Nema17=1 sun_gear=2 housing=3 carrier=4 cover=5

# For this notebook the names of mesh files were changed to numbers eg: obj_000001.ply
# codebase at /captaib_backup/datagen_ws


class Adapt_Dataset():

    def __init__(self, scene_id):
        
        #self.dataset_name = dataset_name
        self.root_dir = root_path
        self.dat_dir = root_path + "/" + scene_id
        self.dir = self.dat_dir 
        self.rgb_dir = self.dir +'/rgb'  
        self.dpt_dir = self.dir+'/depth'
        self.msk_dir = self.dir+'/mask'
        self.segmap_dir = self.dir+'/seg_maps'
        self.kps_dir = self.root_dir+'/keypoints'
        self.model_dir = self.root_dir+'/model_meshes'
        self.pcd_dir = self.root_dir+'/model_pointcloud'
        self.n_keypoints = 8
        self.n_objects = 5
        self.n_sample_points = 8192 + 4096   # 12288

        self.mesh_dict = { }
        self.corners_dict = { }
        self.kps_dict = { }
        self.rgb = None

        self.corners_file = os.path.join(self.kps_dir,'corners.pkl')
        
        if self.n_keypoints == 8:
            kp_type = 'farthest_8'
        else:
            kp_type = 'farthest_{}'.format(self.n_keypoints)
            
    
    def get_cam_info(self,idx):
                 scene_cam_path = os.path.join(self.dir,'scene_camera.json')

                 if os.path.exists(scene_cam_path): 
                    with open(scene_cam_path,"r") as k:
                        for i,j in enumerate(k):
                            im_dict = json.loads(j)
                            if i == idx:
                                this_cam = im_dict
                                
                        cam_K = this_cam[str(idx)]['cam_K']
                        dpt_cam_K = this_cam[str(idx)]['dpt_cam_K']
                        K = np.array(cam_K).reshape(3,3)
                        dpt_K = np.array(dpt_cam_K).reshape(3,3)
                        cam_scale =  this_cam[str(idx)]['depth_scale']

                    return K,dpt_K, cam_scale

                 else:
                      print("missing scene_camera.json :")
                    
    def get_segmap(self,idx):
        segmap = np.load(os.path.join(self.segmap_dir,str(idx).zfill(img_id_leading_zeros)+'_seg_map'+seg_format))
        segmap_flattened = segmap.flatten()
        
        return  segmap_flattened
        
    def get_item(self, idx):
        with open(os.path.join(self.dir, 'cld_rgb_nrms/{}.pkl'.format(idx)),'rb') as f:
                cld_rgb_nrms = pkl.load(f)
                
        labels = np.load(os.path.join(self.dir, 'labels/{}.npy'.format(idx)))
        cld_rgb_nrms = np.asarray(cld_rgb_nrms)
        
        return torch.LongTensor(labels.astype(np.int32)), torch.from_numpy(cld_rgb_nrms.astype(np.float32))
    
    def __len__(self):
        return len(os.listdir(self.rgb_dir))
    
    def __getitem__(self, idx):
        data = self.get_item(idx)
        return data
    
    def get_image(self,idx):
        with Image.open(os.path.join(self.rgb_dir,str(idx).zfill(img_id_leading_zeros)+ rgb_format)) as ri:
                rgb = np.array(ri)[:, :, :3]
                rgb = np.transpose(rgb, (2, 0, 1))
        
        return rgb

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
def generate_kps_corners_pcds():
    dataset = Adapt_Dataset(scene_id='stage_1')
    cls_ids = [i for i in range(1,5)]
    corners = []
    
    
    for cls_id in cls_ids:
        if not os.path.exists(dataset.kps_dir):
            os.makedirs(dataset.kps_dir)

        kps_path = os.path.join(dataset.kps_dir,str(cls_id)+"_fps_{}.".format(dataset.n_keypoints))
        corners_path = os.path.join(dataset.kps_dir,'corners.pkl')
        mesh = o3d.io.read_triangle_mesh(dataset.model_dir +'/obj_'+str(cls_id).zfill(6)+'.ply')
        pcd = mesh.sample_points_poisson_disk(number_of_points=30000)
        

        if not os.path.exists(dataset.pcd_dir):
            os.makedirs(dataset.pcd_dir)

        o3d.io.write_point_cloud(dataset.pcd_dir+'/obj_'+str(cls_id).zfill(6)+'.ply', pcd)

        if not os.path.exists(kps_path):
            print('Writing kps for class '+str(cls_id))
            
            pts = np.asarray(pcd.points)
            tensor_pcd = o3d.t.geometry.PointCloud(pts)
            fps_pcd = tensor_pcd.farthest_point_down_sample(dataset.n_keypoints)
            print("done")
            fps_pcd = fps_pcd.to_legacy()
            o3d.io.write_point_cloud(os.path.join(kps_path+'pcd'), fps_pcd)

            with open(os.path.join(kps_path+'pkl'), 'wb') as file:
                pkl.dump(np.asarray(fps_pcd.points), file)


        # calculate corners
        if not os.path.exists(corners_path):
            print('Writing corners')
            ctrs = copy.deepcopy(pcd).get_minimal_oriented_bounding_box()
            ctrs = np.asarray(ctrs.get_box_points())
            corners.append(ctrs)

    if not os.path.exists(corners_path):
        with open(corners_path, 'wb') as file:
            pkl.dump(corners, file)

In [2]:
# this part runs on pvn3d

#Function to project depth to pointcloud
def dpt_2_cld( depth_frame, K,segMask = None,cam_scale=1):

    w = depth_frame.shape[1]
    h = depth_frame.shape[0]
    
    xmap = np.array([[j for i in range(w)] for j in range(h)])
    ymap = np.array([[i for i in range(w)] for j in range(h)])
    
    dpt = np.array(depth_frame, dtype=np.float32)
    dpt = dpt/1000
    
    if len(dpt.shape) > 2:
            dpt = dpt[:, :, 0]
    msk_dp = dpt > -1
    choose = msk_dp.flatten().nonzero()[0].astype(np.uint32)

    if len(choose) < 1:
        return None, None
        
    dpt_mskd = dpt.flatten()[choose][:, np.newaxis].astype(np.float32)
    xmap_mskd = xmap.flatten()[choose][:, np.newaxis].astype(np.float32)
    ymap_mskd = ymap.flatten()[choose][:, np.newaxis].astype(np.float32)
    cam_cx, cam_cy = K[0][2], K[1][2]
    cam_fx, cam_fy = K[0][0], K[1][1]
    
    if segMask is not None:
             
        focus_points = np.argwhere(segMask != 0)
        focus = segMask != 0
        
        # projecting only the focus
        pt2 = dpt_mskd[focus.flatten()] / cam_scale
        pt0b= (ymap_mskd[focus.flatten()] - cam_cx) * pt2 / cam_fx
        pt1b= (xmap_mskd[focus.flatten()] - cam_cy) * pt2 / cam_fy
        focus_points = np.concatenate((pt0b, pt1b, pt2),axis=1)
               
        return focus_points , choose
    
    else :

        # projecting the cloud as a whole    
        pt2 = dpt_mskd / cam_scale
        pt0 = (ymap_mskd - cam_cx) * pt2 / cam_fx
        pt1 = (xmap_mskd - cam_cy) * pt2 / cam_fy
        cld = np.concatenate((pt0, pt1, pt2),axis=1)
        
        return cld , choose

    
# Lets do cld_rgb_nrms itself instead of cld_choose_nrms!

import pcl
from PIL import Image


from lib.utils.basic_utils import Basic_Utils
from common import Config
config = Config(dataset_name='Adapt')
bs_utils = Basic_Utils(config)

dataset = Adapt_Dataset(scene_id='stage_2')
cld_rgb_nrms_path = os.path.join(dataset.dat_dir, 'cld_rgb_nrms')


def get_normal( cld):
        cloud = pcl.PointCloud()
        cld = cld.astype(np.float32)
        cloud.from_array(cld)
        ne = cloud.make_NormalEstimation()
        kdtree = cloud.make_kdtree()
        ne.set_SearchMethod(kdtree)
        ne.set_KSearch(50)
        n = ne.compute()
        n = n.to_array()
        return n

rgb_directory = os.path.join(dataset.dir,'rgb')
for filename in os.listdir(rgb_directory):
    #print("filename",filename)
    name, extension = os.path.splitext(filename)
    #print("name", name)
    idx = name.lstrip('0')
    idx = int(idx) if idx else 0

    K,dpt_K, cam_scale = dataset.get_cam_info(idx)
    segmap_flattened = dataset.get_segmap(idx)

    with Image.open(os.path.join(dataset.dpt_dir,str(idx).zfill(img_id_leading_zeros)+ dpt_format)) as di:
        dpt = np.array(di)
        
    with Image.open(os.path.join(dataset.rgb_dir,str(idx).zfill(img_id_leading_zeros)+ rgb_format)) as ri:
                rgb = np.array(ri)[:, :, :3]
                rgb = np.transpose(rgb, (2, 0, 1))


    #Back-projection util function
    cld, choose = dpt_2_cld(dpt, dpt_K)
    #choose = choose.reshape(choose.shape[0],1)
    #print("choose :", np.shape(choose))
    
    segmap_flattened = segmap_flattened[choose]

    # sample only 12288 points and get the respective cld_rgb_nrm + labels
    rgb_lst = []
    for ic in range(rgb.shape[0]):
        rgb_lst.append(
            rgb[ic].flatten()[choose].astype(np.float32)
        )
     
    rgb_pt = np.transpose(np.array(rgb_lst), (1, 0)).copy()
    #print("rgb :",np.shape(rgb_pt))
    
    choose = np.array([choose], dtype= np.uint32)
    choose_2 = np.array([i for i in range(len(choose[0, :]))])
    
    
    if len(choose_2) < 400:
        print("not_enough points")
        
    if len(choose_2) > config.n_sample_points:
        c_mask = np.zeros(len(choose_2), dtype=int)
        c_mask[:config.n_sample_points] = 1
        np.random.shuffle(c_mask)
        choose_2 = choose_2[c_mask.nonzero()]
    else:
        choose_2 = np.pad(choose_2, (0, dataset.n_sample_points-len(choose_2)), 'wrap')
    
    #print("choose_2 :", np.shape(choose_2))
    
    
    rgb_pt = rgb_pt[choose_2,:]
    #print("rgb :", np.shape(rgb_pt))
    
    cld = cld[choose_2, :]
    #print("cld :", np.shape(cld))
    normals = get_normal(cld)[:,:3]
    normals[np.isnan(normals)] = 0.
    #print("normals :",np.shape(normals))
    
    cld_rgb_nrm = np.concatenate((cld, rgb_pt, normals), axis=1)
    
    
    #cld_rgb_nrm = cld_rgb_nrm[choose_2, :]
    
    # labels are the corresponding per sampled point, segmap is the perpoint labels for all points
    labels = segmap_flattened[choose_2].astype(np.int32)
    
    #save the choose with each label 
    choose = choose[:, choose_2]
    arr = [[labels],[choose]]
    #print("arr", arr)
    
    
    label_path = os.path.join(dataset.dat_dir, 'labels')
    if not os.path.exists(label_path):
        os.makedirs(label_path)
    if not os.path.exists(os.path.join(label_path,str(idx)+'.npy')):
        np.save(os.path.join(label_path,str(idx)+'.npy'), labels)  # NOTE! Only label is saved. you can save arr next time!
    #all_arr = np.concatenate( (cld, choose.reshape(choose.shape[0],1), normals[:,:3]) , axis = 1)
    
    if not os.path.exists(cld_rgb_nrms_path):
        os.makedirs(cld_rgb_nrms_path)
    
    if not os.path.exists(os.path.join(cld_rgb_nrms_path,str(idx)+'.pkl')):
        print('Writing file '+str(idx)) 
        with open(os.path.join(cld_rgb_nrms_path,str(idx)+'.pkl'), 'wb') as file:
            pkl.dump(cld_rgb_nrm,file)

            
            
# segmap and cld_rgb_nrms have to be downsampled!

ModuleNotFoundError: No module named 'pcl'